In [1]:
from google.cloud import bigquery
from dotenv import load_dotenv
import json
import os
import pandas as pd
from pyoso import Client
import requests

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
AGORA_API_KEY = os.environ['AGORA_API_KEY']
GCP_PROJECT = 'opensource-observer'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../../../oso_gcp_credentials.json'

client = Client(api_key=OSO_API_KEY)
bq_client = bigquery.Client(GCP_PROJECT)

In [2]:
def stringify(arr):
    return "'" + "','".join(arr) + "'"

# Lookup addresses

In [11]:
def fetch_delegates():
    url = 'https://vote.optimism.io/api/v1/delegates'
    params = {'limit': 100, 'offset': 0, 'sort': 'most_delegators'}
    headers = {
        'accept': 'application/json',
        'Authorization': f'Bearer {AGORA_API_KEY}'
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        agora_data = response.json()
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.text)
    return agora_data

delegate_result = fetch_delegates()
delegates = [x['address'].lower() for x in delegate_result['data']]
print(len(delegates))

100


In [12]:
bq_result = bq_client.query(f"SELECT DISTINCT LOWER(address) FROM `{GCP_PROJECT}.op_atlas.badgeholder`")
badgeholders = [x[0] for x in bq_result.result()]
print(len(badgeholders))

132


In [13]:
ADDRESSES = list(set(delegates + badgeholders))
print(len(ADDRESSES))

214


# Lookup project users

In [42]:
df = client.to_pandas(f"""

    WITH filtered_events AS (
      SELECT
        from_artifact_name,
        to_artifact_id
      FROM int_events__blockchain
      WHERE
        time >= DATE '2024-10-01'
        AND from_artifact_name IN ({stringify(ADDRESSES)})
        AND event_source != 'WORLDCHAIN'
    ),
    project_users AS (
      SELECT
        abp.project_id,
        COUNT(DISTINCT e.from_artifact_name) AS num_addresses
      FROM filtered_events AS e
      JOIN artifacts_by_project_v1 AS abp ON e.to_artifact_id = abp.artifact_id
      WHERE abp.project_source = 'OP_ATLAS'
      GROUP BY 1
    ),
    worldchain_projects AS (
      SELECT
          project_id,
          COUNT(DISTINCT artifact_id) AS num_addresses
      FROM artifacts_by_project_v1
      WHERE artifact_source = 'WORLDCHAIN'
      GROUP BY 1
    )
    SELECT
      p.project_id AS oso_project_id,
      p.project_name AS atlas_project_id,
      p.display_name AS project_name,
      COALESCE(project_users.num_addresses, 0) AS num_addresses,
      COALESCE(worldchain_projects.num_addresses, 0) AS worldchain_addresses
    FROM projects_v1 AS p
    JOIN projects_by_collection_v1 AS pbc ON p.project_id = pbc.project_id
    LEFT JOIN project_users ON p.project_id = project_users.project_id
    LEFT JOIN worldchain_projects ON p.project_id = worldchain_projects.project_id
    WHERE pbc.collection_name = '8-1'

""")

In [44]:
#df.to_csv("interactions_with_project.csv")